In [ ]:
import pandas as pd
import numpy as np

# Method Chaining

In [ ]:
sales = pd.read_csv('sales_week4.csv')

In [ ]:
sales.head()

## Typical Workflow

In [ ]:
only_750 = sales.loc[0:750]
group_by_Type = only_750.groupby('Type')
agg = group_by_Type.agg({'Units':'mean', 'Sales':'sum'})
agg.sort_values('Sales', ascending=False)

## Chain Method

In [ ]:
(sales
.loc[0:750]
.groupby('Type')
.agg({'Units':'mean', 'Sales':'sum'})
.sort_values('Sales', ascending=False)
)

In [ ]:
print(sales.Units.isna().sum())
print(sales.Units.fillna(200).isna().sum())

# Date and Time
Pandas uses special methods and data types to handle date and time.  Using these methods/data-types allows you to manipulate dates/time in unique ways.  You are able to subset and filter data based on date/time.  You are also able to aggregate data into time windows such as days, quarters, and years.
* Epoch is January 1, 1970

##  There are many abbreviations that represent date/time  frequency
Documentation:  [DateOffsets](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases)

<div align=left>

    
|letter|Object|
|------|------|
|M|Month|
|D|Day|
|Q|Quarterly|
|H|Hourly|
|W|Weekly|
|Y|Yearly|
|AS|YearStart|
    
    
</div>




## Convert column to datetime data type
### Pandas `to_datetime` function
This comes in useful for converting string columns in DataFrames to dates.
* to_datetime can transform Series

In [ ]:
# Solution 1: use to_datetime
crime = pd.read_csv('bmore_crime.csv')
crime.info()

In [ ]:
# Solution 1: use to_datetime
crime['CrimeDateTime'] = pd.to_datetime(crime['CrimeDateTime'])
crime.info()

### Convert column by using the parse_dates parameter

In [ ]:
# Solution 2:   use parse_dates named parameter
crime = pd.read_csv('bmore_crime.csv', parse_dates=['CrimeDateTime'])
crime.info()

## Filtering columns with time data

### Select all the rows in the CrimeDateTime column that have a certain date-time value

In [ ]:
crime[crime.CrimeDateTime == '2021-03-05 11:03:00']

### Partial match with `.between()` method
Allows you to do all the string searches above

In [ ]:
crime[crime.CrimeDateTime.between('2021-01-01', '2021-01-02')].head()

### To include both dates in `.between()` method  You must specify the end of the day

In [ ]:
crime[crime.CrimeDateTime.between('2021-3-4', '2021-3-5 23:59:59')].shape

### Filter dates with conditional expressions (VERY IMPORTANT!)

In [ ]:
crime_jan1_mar6 = crime[(crime.CrimeDateTime >= '2021-01-01 22') & (crime.CrimeDateTime <= '2021-03-06 11:22:00')]

In [ ]:
crime_jan1_mar6.groupby(['District', 'Description'])['Total_Incidents'].agg('sum')

## Slicing time series intelligently
We've covered dataframe selection and slicing before.
We will introduce the concept of the DatetimeIndex.  We will examine
the Baltimore crimes dataset.  

In [ ]:
crime = crime.set_index('CrimeDateTime')
print(crime.index[:2])

## DatetimeIndex
When we moved the CrimeDateTime column to the index with `set_index()` method we created a `DatetimeIndex`

In [ ]:
crime.head()

### Use `.loc` to select all the rows equal to a single index

In [ ]:
crime.loc['2021-03-05 23:22:00']

### Select all rows that partially match an index value

In [ ]:
crime.loc['2021-03-05'].head(3)

### Select all rows for a single month or year or hour

In [ ]:
crime.loc['2021-03'].shape

In [ ]:
crime.loc['2021'].shape

In [ ]:
crime.loc['2021-03-05 15'].shape

### Contain the name of the month

In [ ]:
crime.loc['Dec 2020'].sort_index()

In [ ]:
crime.loc['2014-03-06':'2021-03-05'].sort_index()

## Using methods that only work with a DatetimeIndex

In [ ]:
type(crime.index)

### Use `.between_time()` method
Select all crimes that occurred between 2 A.M. and 5 A.M. regardless of the date

In [ ]:
crime.between_time('2:00', '5:00', include_end=False)

### Select all dates at a specific time with `.at_time`

In [ ]:
crime.at_time('5:47').head()

### `first()` methods allow for selecting the first n segments of time

In [ ]:
crime.first('5D').head()

In [ ]:
crime.first('3QS').head()

## Counting the number of weekly crimes
We will use grouping according to some period of time to answer this question.

In [ ]:
crime.groupby(pd.Grouper(freq='W')).size()

## Aggregating weekly homicides and arsons separately
1. Group by each quarter then sum the IS_HOMICIDE and IS_ARSON columns for each group
2. use resample method
3. use groupby method

In [ ]:
crime.groupby(pd.Grouper(freq='Q'))[['IS_HOMICIDE', 'IS_ARSON']].sum().head()

In [ ]:
# Plot Quarterly Frequency of Homicides and Arson
crime.groupby(pd.Grouper(freq='Q'))[['IS_HOMICIDE', 'IS_ARSON']].sum().plot(title='Baltimore Homicides and Arson', 
                                                                            color=['black', 'blue']);


## Grouping by a Timestamp and another column

In [ ]:
employee = pd.read_csv('employee.csv', parse_dates=['JOB_DATE', 'HIRE_DATE'], index_col='HIRE_DATE')
employee.head()

### Let's first do a grouping by just gender, and find the average salary for each

In [ ]:
employee.groupby('GENDER')['BASE_SALARY'].mean().round(-2)

###  Let's find the average salary based on hire date, and group everyone into 10-year buckets:

In [ ]:
employee.resample('10AS')['BASE_SALARY'].mean().round(-2)

### Let's group by gender and a 10-year time span

In [ ]:
employee.groupby('GENDER').resample('10AS')['BASE_SALARY'].mean().round(-2)

In [ ]:
## Place both 10-year grouping and Gender within groupby
employee.groupby(['GENDER', pd.Grouper(freq='10AS')])['BASE_SALARY'].mean().round(-2)

## Pair Programming
1. Using `date_demo.csv` perform the following tasks
* What is the shape of the data in the CSV file?
* What is the min and max dates in the `birth_date` column?
* How many people were born between 7/1/2019 & 7/1/2020? ie filter birth_date column and report number of rows

2. Using `daily-min-temperatures.csv` which represents daily minimum temperature checks perform the following tasks
* Convert `Date` column to datetime dtype
* Create a new DataFrame that only contains data from 1982
* Set index of new DataFrame to `Date` column
* Group DataFrame by Weekly frequency then aggregate the `Temp` column by the aggregating function `mean`


# Data Wrangling
Data wrangling, sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics. The goal of data wrangling is to assure quality and useful data

## dropna()

In [ ]:
sales = pd.read_csv('sales_week4.csv')

In [ ]:
sales.info()

In [ ]:
# Drop all rows that have nan values
sales.dropna(axis=0)

In [ ]:
# drop only rows that have nan values in the Units column
sales.dropna(subset=['Units'], axis=0)

In [ ]:
# drop all columns that contain nan values
sales.dropna(axis=1)

## duplicated() and drop_duplicates()

In [ ]:
sales.duplicated().sum()

In [ ]:
sales.drop_duplicates()

## fillna()

In [ ]:
sales.isna().sum()

In [ ]:
sales.fillna(200).isna().sum()

## String methods

### contains

In [ ]:
sales.head(1)

In [ ]:
sales[sales.Type.str.contains('Men')]

### split

In [ ]:
sales[['Type', 'Department']] = sales.Type.str.split(expand=True)
sales

### upper

In [ ]:
sales['Type'] = sales.Type.str.upper()
sales.Type.str.isupper().sum()

# Pivot Tables

In [ ]:
# insert code
sales = pd.read_excel('https://github.com/datagy/pivot_table_pandas/raw/master/sample_pivot.xlsx',
                      engine='openpyxl', parse_dates=['Date'])
sales.head()

In [ ]:
# Pivot with Region index and mean sales
pivot_table1 = pd.pivot_table(sales, index='Region', values='Sales')
pivot_table1

In [ ]:
# Pivot with Region index and sum of sales
pivot_table2 = pd.pivot_table(sales, index='Region', values='Sales', aggfunc='sum')
pivot_table2

In [ ]:
# Pivot with Dual indices and sum of sales
pivot_table3 = pd.pivot_table(sales, index=['Region', 'Type'], values='Sales', aggfunc='sum')
pivot_table3

In [ ]:
# Pivot with columns and sum of sales
pivot_table3 = pd.pivot_table(sales, index='Type', columns='Region', values='Sales', aggfunc='sum')
pivot_table3